2. (3 puntos) Hacer una función que diseñe filtros notch ideales, de Gauss o de Butterworth.

In [ ]:
def kernel_gaussiano(P, Q, sigma, pasa = 0): #Pasar a meshgrid
    """pasa = 0 es pasabajos. pasa = 1 es pasaaltos. No me hago responsable del resto"""
    kernel = np.zeros((P, Q))
    
    for u in range(P):
        for v in range(Q):
            D_cuadrada = (u - 0.5 * P)**2 + (v - 0.5 * Q)**2
            kernel[u][v] = pasa + (-1)**pasa * exp(- 0.5 * D_cuadrada / sigma**2)
                
    return kernel

In [ ]:
def filtrar_notch(img, tipo = 0, pasa = 0, centro = (0, 0), radio = 1, d0, n = 0.0):
    """Filtro notch. 
    tipo = 0 para ideal, 1 para gaussiano y cualquier otro valor para butterworth.
    pasa = 0 para notchreject, 1 para notchpass.
    centro y radio son los del notch. notch simétrico automático.
    Especificar n solo para butterworth"""
    
    P, Q = img.shape
    
    if tipo == 0:
        kernel = kernel_ideal(P, Q, pasa, centro, radio, d0)
    else if tipo == 1:
        kernel = kernel_gaussiano(P, Q, pasa, centro, radio, d0)
    else:
        kernel = kernel_butterworth(P, Q, pasa, centro, radio, d0, n)
        
    transformada = np.fft.fftshift(np.fft.fft2(img))
    aplico_filtro = kernel * transformada
    img_filtrada = np.real(np.fft.ifft2(np.fft.ifftshift(aplico_filtro)))
    
    return img_filtrada